<a href="https://colab.research.google.com/github/ogyogy/colaboratory/blob/main/ajagekar2020quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ajagekar2020quantum
[PDF](https://arxiv.org/ftp/arxiv/papers/1910/1910.13045.pdf)

## モジュールのインストール

In [ ]:
pip install openjij pyqubo pulp

## モジュールのインポート

In [2]:
import openjij as oj
import pandas as pd
import pulp
from pyqubo import Array
import numpy as np

## 例題

例題として、以下のパラメータを設定する。

| | $i1$  | $i2$ | $i3$ | $i4$ | $i5$ |
| :---: | :---: | :---: | :---: | :---: | :---: |
| $m1$ | $P = 8, C = 1$  | $P = 5, C = 1$  | $P = 9, C = 2$ | $P = 4, C = 3$ | $P = 9, C = 2$ |
| $m2$ | $P = 3, C = 2$  | $P = 3, C = 2$  | $P = 4, C = 3$ | $P = 9, C = 1$ | $P = 4, C = 1$ |
| $m3$ | $P = 6, C = 3$  | $P = 10, C = 1$ | $P = 6, C = 1$ | $P = 11, C = 2$ | $P = 4, C = 1$ |

| | $i1$ | $i2$ | $i3$ | $i4$ | $i5$ |
| :---: | :---: | :---: | :---: | :---: | :---: |
| $R$ | $2$ | $1$ | $3$ | $4$ | $5$ |
| $D$ | $5$ | $8$ | $10$ | $12$ | $10$ |

ここで、$i$はジョブ、$m$はマシン、$P$は処理時間、$C$はコスト、$R$はリリース日、$D$は締切日を表す。

このとき、期待される解は以下のように表される。

<table>
    <thead>
        <tr>
            <th></th>
            <th>$t0$</th>
            <th>$t1$</th>
            <th>$t2$</th>
            <th>$t3$</th>
            <th>$t4$</th>
            <th>$t5$</th>
            <th>$t6$</th>
            <th>$t7$</th>
            <th>$t8$</th>
            <th>$t9$</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>$m1$</td>
            <td></td>
            <td>$i2$</td>
            <td>$i2$</td>
            <td>$i2$</td>
            <td>$i2$</td>
            <td>$i2$</td>
            <td>$i4$</td>
            <td>$i4$</td>
            <td>$i4$</td>
            <td>$i4$</td>
        </tr>
        <tr>
            <td>$m2$</td>
            <td></td>
            <td></td>
            <td>$i1$</td>
            <td>$i1$</td>
            <td>$i1$</td>
            <td>$i5$</td>
            <td>$i5$</td>
            <td>$i5$</td>
            <td>$i5$</td>
            <td></td>
        </tr>
        <tr>
            <td>$m3$</td>
            <td></td>
            <td></td>
            <td></td>
            <td>$i3$</td>
            <td>$i3$</td>
            <td>$i3$</td>
            <td>$i3$</td>
            <td>$i3$</td>
            <td>$i3$</td>
            <td></td>
        </tr>
    </tbody>
</table>

ここで、$t$は処理時間を表す。

## パラメータの設定

In [3]:
# ジョブの数
I = 5
# マシンの数
M = 3

In [4]:
# ジョブiをマシンmで行うのにかかる時間P[i][m]
P = [
     [8, 3, 6],
     [5, 3, 10],
     [9, 4, 6],
     [4, 9, 11],
     [9, 4, 4]
]

In [5]:
# コストC[i][m]
# 横軸はマシンm、縦軸はジョブi
C = [
     [1, 2, 3],
     [1, 2, 1],
     [2, 3, 1],
     [3, 1, 2],
     [2, 1, 1]
]

In [6]:
# リリース日
R = [2, 1, 3, 4, 5]
# 締切日
D = [5, 8, 10, 12, 10]

In [7]:
# 各ジョブの最大処理時間の和を取得 (14)
U = sum(max(P[i]) for m in range(M) for i in range(I))

## Relaxed MILPモデルの定式化

In [8]:
# MILPモデルの初期化
prob = pulp.LpProblem('MILP', pulp.LpMinimize)

# ジョブiがマシンmに割り当てられているかを表すバイナリ変数x[i][m]
# (17)、(19) におけるx[i][m] ∈ {0, 1} が実質定義される
x = [[pulp.LpVariable('x_{}_{}'.format(i, m), cat='Binary') for m in range(M)] for i in range(I)]
    
# 各マシンmでのジョブiの開始日ts[i]
# 引数lowBoundにより、(17)、(19) におけるts[i] >= 0 が実質定義される
ts = [pulp.LpVariable('ts_{}'.format(i), lowBound=0, cat='Integer') for i in range(I)]

# 目的関数(objective function)
obj = pulp.lpSum([C[i][m] * x[i][m] for i in range(I) for m in range(M)])

# 目的関数を追加
prob += obj

# 制約条件を追加 (19)
# ジョブに設定されたリリース日以降にジョブを開始 (10)
for i in range(I):
    prob += ts[i] >= R[i]

# ジョブに設定された締切日に間に合うようにジョブを開始 (11)
for i in range(I):
    prob += ts[i] <= D[i] - pulp.lpSum([P[i][m] * x[i][m] for m in range(M)])

# 各ジョブは1つのマシンのみで処理 (12)
for i in range(I):
    prob += pulp.lpSum([x[i][m] for m in range(M)]) == 1

 ## Hybrid QC-MILP Decomposition Methodの実行

In [9]:
# 最適解が得られたか
opt = False
# ループ回数をカウント
iter = 0
# QC stepで問題を解く回数
num_reads = 10

while not opt:
    iter += 1

    print('=====iter {}====='.format(iter))

    # Relaxed MILP problemを解く
    status = prob.solve()

    # 実行結果の状態を文字列として取得
    # 解が得られた場合はOptimalが取得される
    # 文字列がOptimal以外(Infeasible, Infeasible, Unbounded, Undefined)の場合は制約条件を見直す
    status_str = pulp.LpStatus[status]

    print('MILP status = {}'.format(status_str))

    print('-----MILP problem solution-----')
    for v in prob.variables():
        print('{} = {}'.format(v.name, v.varValue))

    # Relaxed MILP problemの解が得られていればQC stepを実行
    if pulp.LpStatus[status] == 'Optimal':
        # Relaxed MILP problemの解を用いてQCで計算
        # 同じマシンに割り当てられたジョブの順序を表すバイナリ変数y[i][j]
        y = Array.create('y', shape=(I, I), vartype='BINARY')

        # 集合Sを生成 (20)
        S = []
        for i in range(I):
            for j in range(I):
                if i != j:
                    for m in range(M):
                        if x[i][m].value() == 1 and x[j][m].value() == 1:
                            S.append((i, j))

        # 目的関数を定義 (20)
        # 今回は目的関数がそのままハミルトニアンになる
        H = sum((1 - y[i][j] - y[j][i] + 2 * y[i][j] * y[j][i] + y[i][j] *
                    (U * (ts[i].value() - ts[j].value()) +
                    sum(
                        (P[i][m] * x[i][m].value()) for m in range(M)
                    ))
                ) for (i, j) in S)
        
        # モデルをコンパイル
        model = H.compile()
        qubo, offset = model.to_qubo()

        # SQAを用いる
        sampler = oj.SQASampler()

        # SamplerにQUBOを渡す
        response = sampler.sample_qubo(Q=qubo, num_reads=num_reads)

        print('-----QC step solution-----')
        print(response.first.sample)
        print('----------')

        # ガントチャートをm行t列で生成
        gantt_chart = [['' for ts in range(max(D))] for m in range(M)]

        # QCの解をデコード
        decoded_sample = model.decode_sample(response.first.sample, vartype="BINARY")

        # 同じマシンに2つ以上ジョブが割り当てられている場合は実行可能かどうかチェック
        seq_vars = [(i, j) for i, j in S if decoded_sample.array('y', (i, j)) == 1]

        # 各ジョブの処理時間と割り当てられたマシンを取得
        process_times = []
        machines = []
        for i in range(I):
            for m in range(M):
                if x[i][m].value() == 1:
                    process_times.append(P[i][m])
                    machines.append(m)

        # ジョブが追加されたか
        is_added = [False] * I

        # 整数カットの対象となるマシンか
        is_cut = [False] * M

        # QC stepで求めたジョブの順序に基づきガントチャートにジョブを追加
        # Relaxed MILPで求めたts[i]は順序を考慮していないのでジョブが同じ時間で重なる場合がある
        # 同じ時間でジョブが重なる場合、後に実行されるジョブを先に実行したジョブの直後にスライドする
        for i, j in seq_vars:
            try:
                if not is_added[i]:
                    job_i_start_time = int(ts[i].value())
                    job_i_end_time = job_i_start_time + process_times[i]
                    job_i_machine = machines[i]

                    # ジョブiの開始時刻と終了時刻が条件を満たしていなければ整数カットの対象に追加
                    if job_i_start_time < R[i] or job_i_end_time > D[i]:
                        is_cut[job_i_machine] = True
                    
                    # ジョブiを開始時刻から埋める
                    for t in range(job_i_start_time, job_i_end_time):
                        gantt_chart[job_i_machine][t] = 'j{}'.format(i)

                    # ジョブiを追加済みに登録
                    is_added[i] = True

                if not is_added[j]:
                    job_j_start_time = int(ts[j].value())
                    job_j_machine = machines[i]

                    # ジョブjの開始時刻が重ならないかチェック
                    if gantt_chart[job_j_machine][job_j_start_time] != '':
                        # すでにガントチャートにジョブが割り当てられていれば開始時間をジョブiの直後までずらす
                        # 直前のジョブiはjob_i_end_time - 1まで割り当てられている
                        job_j_start_time = job_i_end_time

                    job_j_end_time = job_j_start_time + process_times[j]

                    # ジョブjの開始時刻と終了時刻が条件を満たしていなければ整数カットの対象に追加
                    if job_j_start_time < R[j] or job_i_end_time > D[j]:
                        is_cut[job_j_machine] = True

                    # ジョブjを開始時刻から埋める
                    for t in range(job_j_start_time, job_j_end_time):
                        gantt_chart[job_j_machine][t] = 'j{}'.format(j)

                    # ジョブjを追加済みに登録
                    is_added[j] = True
            except IndexError:
                # 最大時間をはみ出た場合は解が得られなかったとする
                is_cut[job_i_machine] = True

        # 整数カットの対象となるマシンが存在しなければ最適解が得られたとする
        if True not in is_cut:
            opt = True

        # 1つのマシンに1つしか割り当てられていないジョブを追加
        for i in range(I):
            if not is_added[i]:
                job_i_start_time = int(ts[i].value())
                job_i_end_time = job_i_start_time + process_times[i]
                job_i_machine = machines[i]
                for t in range(job_i_start_time, job_i_end_time):
                    gantt_chart[job_i_machine][t] = 'j{}'.format(i)
        
        # 最適解が得られたか表示
        print('opt = {}'.format(opt))

        # Relaxed MILPに整数カットを追加
        # 同じ組み合わせの割り当てを除外
        for m in range(M):
            if is_cut[m]:
                S_ = [i for i in range(I) if x[i][m].value() == 1]
                prob += sum(x[i][m] for i in S_) <= len(S_) - 1
    else:
        # 解が存在しない場合は終了
        print('No solution exists.')
        print('Stop procedure and exit the loop.')
        break

=====iter 1=====
MILP status = Optimal
-----MILP problem solution-----
ts_0 = 2.0
ts_1 = 1.0
ts_2 = 3.0
ts_3 = 4.0
ts_4 = 5.0
x_0_0 = 0.0
x_0_1 = 1.0
x_0_2 = 0.0
x_1_0 = 1.0
x_1_1 = 0.0
x_1_2 = 0.0
x_2_0 = 0.0
x_2_1 = 0.0
x_2_2 = 1.0
x_3_0 = 1.0
x_3_1 = 0.0
x_3_2 = 0.0
x_4_0 = 0.0
x_4_1 = 0.0
x_4_2 = 1.0
-----QC step solution-----
{'y[1][3]': 1, 'y[2][4]': 1, 'y[3][1]': 0, 'y[4][2]': 0}
----------
opt = False
=====iter 2=====
MILP status = Optimal
-----MILP problem solution-----
ts_0 = 2.0
ts_1 = 1.0
ts_2 = 3.0
ts_3 = 4.0
ts_4 = 5.0
x_0_0 = 0.0
x_0_1 = 1.0
x_0_2 = 0.0
x_1_0 = 1.0
x_1_1 = 0.0
x_1_2 = 0.0
x_2_0 = 0.0
x_2_1 = 0.0
x_2_2 = 1.0
x_3_0 = 1.0
x_3_1 = 0.0
x_3_2 = 0.0
x_4_0 = 0.0
x_4_1 = 1.0
x_4_2 = 0.0
-----QC step solution-----
{'y[0][4]': 1, 'y[1][3]': 1, 'y[3][1]': 0, 'y[4][0]': 0}
----------
opt = True


## ガントチャートの出力

例題で示した期待される解と異なり、ジョブとマシンの添え字が0から始まることに注意する。

In [10]:
# ガントチャートをDateFrameに変換
df = pd.DataFrame(gantt_chart,
                  index=['m' + str(m) for m in range(M)],
                  columns=['t' + str(t) for t in range(max(D))])
# ガントチャートを表示
# ジョブとマシンの添え字が0から始まることに注意
df

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11
m0,,j1,j1,j1,j1,j1,j3,j3,j3,j3,,
m1,,,j0,j0,j0,j4,j4,j4,j4,,,
m2,,,,j2,j2,j2,j2,j2,j2,,,
